# Obstacle avoidance block

## Imports

Import the script `thymio.py` and other libraries 

In [24]:
import os
import sys
import time
import serial
import math

# Variable to know if Thymio has already been connected before or not (== 1 if it is the case)
try:
    CONNECC
except NameError:
    CONNECC = 0
    print('Thymio will be connected.')

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
#sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

# Print the path to Python3 executable
# print(sys.executable)

## Thymio connection

First connect Thymio to the computer on one USB port through the USB cable.

### Look-up table for ports:
Replace `/dev/cu.usbmodem142101` (or current port value) below by the correct computer port:

Lucas: `\\.\COM3` for USB-cable and `\\.\COM4` for USB-dongle

Emma: ` `

Elise: ` `

Océane: ` `

In [2]:
if CONNECC == 0:
    CONNECC = 1
    th = Thymio.serial(port="\\.\COM3", refreshing_rate=0.1)
    time.sleep(1)

## Tests

Testing communication with Thymio.

In [3]:
# Defines if tests should be run
TEST = 1

### Retrieve values

In [5]:
if TEST == 1:
    for i in range(10):
        print(th["prox.horizontal"])
        time.sleep(0.2)

[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1602, 2638]
[0, 0, 0, 0, 0, 2528, 4394]
[0, 0, 0, 0, 0, 2548, 4362]
[0, 0, 0, 0, 0, 2673, 4432]
[0, 0, 0, 0, 0, 2817, 4350]
[0, 0, 0, 0, 0, 3097, 4486]
[0, 0, 0, 0, 0, 2778, 4477]


### Set values

In [ ]:
if TEST == 1:
    th.set_var("motor.left.target", 100)
    th.set_var("motor.right.target", 100)

    time.sleep(1)

    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)

## Obstacle avoidance algorithm

The robot can be in two states: global navigation or obstacle avoidance when one is detected by the sensors (caution for not confounding walls from the map and unknown obstacles).

When in obstacle avoidance state, the robot accelerates the wheels close to the obstacles.

In [ ]:
# Tests
# th.variable_description()

In [82]:
# Tests

#dir(th)
#th.variable_description()
# Conio
# th.set_var("mic.intensity", 0)

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
print(th["prox.horizontal"][0])
print(th["prox.horizontal"][4])

3739
0


In [18]:
State = 0 # State == 0 => Thymio goes straight, State == 1 => Thymio is in obstacle avoidance mode
Velocity_0 = 100
Obstacle_gain = 5
Threshold_enter_obstavoid = 20 # Threshold to enter obstacle avoidance mode
Threshold_quit_obstavoid = 10 # Threshold to quit obstacle avoidance mode
Timer = 1 * 10**(-3) # 1ms
Counter = 0
Limit_loops = 1000 # number of while loops to go through before quitting, entering obstacle avoidance mode reset the counter

while Counter < Limit_loops:
    if (State == 0 and
        (th["prox.horizontal"][0] > Threshold_enter_obstavoid or
        th["prox.horizontal"][4] > Threshold_enter_obstavoid)): # enter obstacle avoidance
        State = 1
    elif (State == 1 and
        th["prox.horizontal"][0] < Threshold_quit_obstavoid and
        th["prox.horizontal"][4] < Threshold_quit_obstavoid): # quit obstacle avoidance
        State = 0
    
    if State == 0: # goes straight
        th.set_var("motor.right.target", Velocity_0)
        th.set_var("motor.left.target", Velocity_0)
        Counter += 1
    else: # avoid obstacle depending on sensors values
        Right_gain = int(Obstacle_gain*(th["prox.horizontal"][4]/100)) # integer value of right wheel gain
        th.set_var("motor.right.target", (Velocity_0 + Right_gain))
        Left_gain = int(Obstacle_gain*(th["prox.horizontal"][0]/100)) #integer value of left wheel gain
        th.set_var("motor.left.target", (Velocity_0 + Left_gain))
        Counter = 0
    
    print(Counter)
    time.sleep(Timer) # wait 1ms before relooping


th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Calibration Odometry

~ 0.0325 cm/s for 1 motor input unit of Thymio

rotation: ~9.32s for 360° with [100, -100] motor inputs => ~ 0.0067 rad/s for [1, -1] motor inputs

In [ ]:
speed = []
omega = 2*3.1415926/9.32
omega_thymio = omega/100
print(omega_thymio)

In [96]:
counter = 0
VelLeft = 100
VelRight = 50
adjust = 1.9 #4.15
SpeedGain = adjust*0.0325 * 10**(-3)
timer = 1 * 10**(-3) # 1ms
b = 0.095 # distance between Thymio's wheels in m

X = 0
Y = 0
Theta = 0

t = 0

while counter < 1000:
    #a = time.time()
    counter += 1
    th.set_var("motor.right.target", VelRight)
    th.set_var("motor.left.target", VelLeft)

    DTheta = (VelLeft*SpeedGain - VelRight*SpeedGain)/(2*b)
    DS = (VelLeft + VelRight)/2
    
    Theta += DTheta
    X += DS*SpeedGain*math.cos(math.radians(Theta))
    Y += DS*SpeedGain*math.sin(math.radians(Theta))

    time.sleep(timer)
    #b = time.time()
    #t += b-a

#print(t)
th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

print(X)
print(Y)
print(Theta)

4.56931858426016
0.6530057740542673
16.24999999999978


In [72]:
t = []
for i in range(1,1000):
    a = time.time()
    time.sleep(0.001)
    b = time.time()
    t.append(b-a)

print(sum(t)/1000)

0.004234578609466553


In [ ]:


delta_threshold = 300
square = 5 # cm

count = 0
current_square = 1
previous_square = current_square
change = []
vel = 100
start_time =  time.time()
#print(th["prox.ground.delta"])

while (count < 3):
    th.set_var("motor.left.target", vel)
    th.set_var("motor.right.target", vel)
    ground = th["prox.ground.delta"]
    if (time.time() - start_time) > 30:
        print('TIMED OUT')
        break
    if (ground[0] < delta_threshold and ground[1] < delta_threshold):
        current_square = 0
    else:
        current_square = 1
    if current_square != previous_square:
        start_time = time.time()
        change.append(time.time())
        previous_square = current_square
        count += 1

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)

first = change[0]
change = [x - first for x in change]
for i in range(0, len(change) - 1):
        speed.append(square/(change[i+1]-change[i])/vel) # [cm/s by Thymio velocity unit]

# print(change)
mean_speed = sum(speed)/len(speed)
print(mean_speed)

In [ ]:
# Rotation odometry

start = time.time()
th.set_var("motor.left.target", 100)
th.set_var("motor.right.target", 2**16 - 100)
time.sleep(1)

while (stop - start) < 9.32:
    stop = time.time()

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
print(stop - start)

In [ ]:
print(th["prox.horizontal"])

In [ ]:
# Initialisation for testing the obstacle avoidance algorithm, Thymio is going straight towards its goal.
Speed0 = 100
ObstacleSpeedGain = 2
ObstacleThresholdLeft = 10
ObstacleThreshholdRight = 20

State = 0
Sensors = [0,0,0,0,0,0,0] # proximity sensors array
Now = time.time() # Actual time in seconds since 1st of Janurary 1970
Timer = Now

while:
    Timer = time.time()

dir